# Make azimuth/height/distance bin files for filter time tests

In [206]:
import itertools
import os
from pathlib import Path
import pandas as pd
import numpy as np

In [207]:
def convert_to_dataframe(bin_path):
    pre_filtered_data = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4) 
    columns = ['x', 'y', 'z', 'intensity']
    df = pd.DataFrame(pre_filtered_data, columns=columns)
    return df

In [208]:
def convert_to_ahd(label_file):
    xyz_format_df = convert_to_dataframe(label_file)
    
    ahd_format_df = pd.DataFrame() 

    x, y, z, intensity = xyz_format_df['x'], xyz_format_df['y'], xyz_format_df['z'], xyz_format_df['intensity']

    # Calculate distance, azimuth, and height
    distance = np.sqrt(x**2 + y**2 + z**2)
    azimuth = np.degrees(np.arctan2(y, x))
    height = np.degrees(np.arctan2(z, np.sqrt(x**2 + y**2)))
    
    ahd_format_df['distance'] = distance
    ahd_format_df['azimuth'] = azimuth
    ahd_format_df['height'] = height
    ahd_format_df['intensity'] = intensity

    return ahd_format_df

In [209]:
def save_as_binary(df, bin_path):
    try:
        # Ensure the DataFrame is in the correct order and data type
        data = df[['azimuth', 'height', 'distance', 'intensity']].astype(np.float32).values
        
        # Write the data to a binary file
        data.tofile(bin_path)
    except:
        print('could not save :' + str(bin_path))

In [210]:
def get_azimuth_height_distance_format(xyz_dir, ahd_dir):
    xyz_dir_path = Path(xyz_dir)
    
    # Make sure taget destination exists
    ahd_dir_path = Path(ahd_dir)
    os.makedirs(ahd_dir_path, exist_ok=True)

    # For each file in source path
    for label_file in xyz_dir_path.iterdir():
    # for label_file in itertools.islice(xyz_dir_path.iterdir(), 1):
        if str(label_file).endswith('.bin'):
            print('.', end='')
            # Convert the file to azimuth, height, distance format
            ahd_format_df = convert_to_ahd(label_file)

            # Create the target file name
            filename = os.path.basename(label_file)
            # Add the file id to the list
            save_path = Path(ahd_dir_path, filename)
            # Save the file in the target directory
            save_as_binary(ahd_format_df, save_path)

In [211]:
xyz_dir_path = '../data/velodyne_points'
ahd_dir_path = '../data/az_height_dist_points'

In [212]:
# Convert data from x, y, z format to azimuth, height, distance format
get_azimuth_height_distance_format(xyz_dir_path, ahd_dir_path)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................